<a href="https://colab.research.google.com/github/sudarshanregmi369/PRODIGY_ML_05/blob/main/Foodrecipe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import models, layers
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score, mean_squared_error

# Step 1: Load and preprocess the data
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = datagen.flow_from_directory(
    'food',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    'food',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# Step 2: Build a CNN for Food Recognition
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

model = models.Sequential()
model.add(base_model)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.4))
model.add(layers.Dense(num_classes, activation='softmax'))

base_model.trainable = False

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(train_generator, steps_per_epoch=len(train_generator),
                    validation_data=validation_generator, validation_steps=len(validation_generator),
                    epochs=10)

# Step 3: Extract features for Calorie Estimation
def extract_features(generator, model):
    features = []
    labels = []

    for _ in range(len(generator)):
        batch, batch_labels = generator.next()
        batch_features = model.predict(batch)
        features.extend(batch_features)
        labels.extend(batch_labels)

    return np.array(features), np.array(labels)

X_train, y_train = extract_features(train_generator, model)
X_val, y_val = extract_features(validation_generator, model)

# Step 4: Build a Regression Model for Calorie Estimation
regression_model = LinearRegression()
regression_model.fit(X_train, y_train)

# Step 5: Evaluate the models
food_recognition_accuracy = model.evaluate(validation_generator)[1]
calorie_predictions = regression_model.predict(X_val)
calorie_mse = mean_squared_error(y_val, calorie_predictions)

print(f"Food Recognition Accuracy: {food_recognition_accuracy}")
print(f"Calorie Estimation MSE: {calorie_mse}")
